# Environment

In [ ]:
!rm -rf waymo-od > /dev/null
!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd waymo-od && git branch -a
!cd waymo-od && git checkout remotes/origin/master
!pip3 install --upgrade pip
!pip3 install waymo-open-dataset-tf-2-1-0==1.2.0 

In [2]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

!mkdir -p data/training
!gcsfuse --only-dir training/ waymo_open_dataset_v_1_2_0_individual_files data/training/

!mkdir -p data/testing
!gcsfuse --only-dir testing/ waymo_open_dataset_v_1_2_0_individual_files data/testing/

In [1]:
import os
import gc
import tensorflow as tf
import math
import numpy as np
import itertools
import matplotlib.pyplot as plt
from google.colab import files
import time

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data_utils
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

from Net_Lib import *

list_train = os.listdir(path='data/training')
list_test = os.listdir(path='data/testing')

# Face Validation

In [ ]:
model_save_name = 'novel_checkpoint.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
checkpoint = torch.load(path, map_location=torch.device('cpu'))
print(checkpoint['loss'])
print(checkpoint['valid'])
print(checkpoint['epoch'])
fig, ax = plt.subplots()
ax.plot(checkpoint['loss'], label="Training")
ax.plot(checkpoint['valid'], label="Validation")
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.legend()
plt.title('Novel Loss on Waymo Subset')

In [ ]:
model_save_name = 'checkpoint.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
checkpoint = torch.load(path, map_location=torch.device('cpu'))
print(checkpoint['loss'])
print(checkpoint['valid'])
print(checkpoint['epoch'])
fig, ax = plt.subplots()
ax.plot(checkpoint['loss'],label="Training")
ax.plot(checkpoint['valid'],label="Validation")
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.legend()
plt.title('YOLO Loss on Waymo Subset')

In [ ]:
model_save_name = 'lstm_checkpoint.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
checkpoint = torch.load(path, map_location=torch.device('cpu'))
print(checkpoint['loss'])
print(checkpoint['valid'])
print(checkpoint['epoch'])
fig, ax = plt.subplots()
ax.plot(checkpoint['loss'],label="Training")
ax.plot(checkpoint['valid'],label="Validation")
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.legend()
plt.title('LSTM Loss on Waymo Subset')

# Get Average YOLO Trajectory Error

In [2]:
# Data extraction and preparation
def create_trajbatch(index,train):
  if(train):
    loc = "data/training/"
    req_list = list_train
  else:
    loc = "data/testing/"
    req_list = list_test

  train_images = []
  train_labels = []
  img_yes = False
  imagenum = 0
  for dataset in list_train[index:index+1]:
    dataset = tf.data.TFRecordDataset(loc+dataset, compression_type='')
    for data in dataset:
      frame = open_dataset.Frame()
      frame.ParseFromString(bytearray(data.numpy()))

      # Get image itself
      for index, image in enumerate(frame.images):
        if(image.name == 1):
          img = tf.image.decode_jpeg(image.image)
          img = tf.image.resize(img, [640, 960])
          train_images.append(img)

      # Get label data
      for cam_labels in frame.projected_lidar_labels:
        if(cam_labels.name != 1):
          continue
        for label in cam_labels.labels:
          train_labels.append(np.array([label.box.center_x//2, label.box.center_y//2,
                                          label.box.width//2, label.box.length//2, label.metadata.speed_x, label.metadata.speed_y, label.metadata.accel_x,
                                            label.metadata.accel_y, label.type, imagenum]))
          img_yes = True
      
      # Write specialized "empty image" output
      if(not img_yes):
        train_labels.append(np.array([-1,-1,-1,-1,-1,-1,-1,-1,-1,imagenum]))
      else:
        img_yes = False
      imagenum += 1
 
  train_labels = torch.from_numpy(np.array(train_labels))
  train_images = torch.from_numpy(np.array(train_images).transpose(0,3,1,2))
  return train_labels, train_images

In [3]:
def create_diffbatch(index, train):
  if(train):
    loc = "data/training/"
    req_list = list_train
  else:
    loc = "data/testing/"
    req_list = list_test

  train_images = []
  train_labels = []
  imagenum = 0
  img_yes = False

  # Variables for tracking to create diffs from same frame context
  prev_context = None
  context = None
  prev_img = None
  img = None
  
  for dataset in req_list[index:index+1]:
    dataset = tf.data.TFRecordDataset(loc+dataset, compression_type='')
    for data in dataset:
      frame = open_dataset.Frame()
      frame.ParseFromString(bytearray(data.numpy()))
      context = frame.context.name
  
      # Indicates new scene
      if(context != prev_context or prev_context == None): 
        for index, image in enumerate(frame.images):
          if(image.name == 1):
            prev_img = tf.image.rgb_to_grayscale(tf.image.decode_jpeg(image.image))
            prev_img = tf.image.resize(prev_img, [640, 960])
        prev_context = context

      # Indicates continuation
      else: 
        for index, image in enumerate(frame.images):
          if(image.name == 1):
            img = tf.image.rgb_to_grayscale(tf.image.decode_jpeg(image.image))
            img = tf.image.resize(img, [640, 960])
            train_images.append(img-prev_img)

        for cam_labels in frame.projected_lidar_labels:
          if(cam_labels.name != 1):
            continue
          for label in cam_labels.labels:
            train_labels.append(np.array([label.box.center_x//2, label.box.center_y//2,
                                          label.box.width//2, label.box.length//2, label.metadata.speed_x, 
                                            label.metadata.speed_y, label.metadata.accel_x,
                                            label.metadata.accel_y, label.type, imagenum]))
            img_yes = True

        if(not img_yes):
          train_labels.append(np.array([-1,-1,-1,-1,-1,-1,-1,-1,-1,imagenum]))
        else:
          img_yes = False
        imagenum += 1

  train_labels = torch.from_numpy(np.array(train_labels))
  train_images = torch.from_numpy(np.array(train_images).transpose(0,3,1,2))
  return train_labels, train_images

In [4]:
# Data extraction and preparation
def create_LSTMbatch(index,train):
  if(train):
    loc = "data/training/"
    req_list = list_train
  else:
    loc = "data/testing/"
    req_list = list_test

  train_images = []
  train_labels = []
  img_yes = False
  imagenum = 0
  for dataset in list_train[index:index+1]:
    dataset = tf.data.TFRecordDataset(loc+dataset, compression_type='')
    for data in dataset:
      frame = open_dataset.Frame()
      frame.ParseFromString(bytearray(data.numpy()))

      # Get image itself
      for index, image in enumerate(frame.images):
        if(image.name == 1):
          img = tf.image.decode_jpeg(image.image)
          img = tf.image.resize(img, [640, 960])
          train_images.append(img)

      # Get label data
      for cam_labels in frame.projected_lidar_labels:
        if(cam_labels.name != 1):
          continue
        for label in cam_labels.labels:
          train_labels.append(np.array([label.metadata.speed_x, label.metadata.speed_y, label.metadata.accel_x,
                                            label.metadata.accel_y, imagenum]))
          img_yes = True
      
      # Write specialized "empty image" output
      if(not img_yes):
        train_labels.append(np.array([-1,-1,-1,-1,imagenum]))
      else:
        img_yes = False
      imagenum += 1
 
  train_labels = torch.from_numpy(np.array(train_labels))
  train_images = torch.from_numpy(np.array(train_images).transpose(0,3,1,2))
  return train_labels, train_images

In [ ]:
anchors = torch.load("anch.pt")
num_anchors = anchors.shape[0]
res = 32
a_cuda=True
H = 1280//(2*res)                   # height of the grid over images
W = 1920//(2*res)                   # width of the grid over images
batchsize=16

network_spec = parse_cfg("yolo.cfg")
module_list = create_network(network_spec, 3)
yolo = Model(module_list).float()
model_save_name = 'checkpoint.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
checkpoint = torch.load(path)
yolo.load_state_dict(checkpoint['model_state_dict'])

network_spec = parse_cfg("novel.cfg")
module_list = create_network(network_spec, 1)
novel = Model(module_list).float()
model_save_name = 'novel_checkpoint.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
checkpoint = torch.load(path)
novel.load_state_dict(checkpoint['model_state_dict'])

model_save_name = 'lstm_checkpoint.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
recurrent = LSTM_module(W*H*50, 360, 2, W*H*5)
checkpoint = torch.load(path)
recurrent.load_state_dict(checkpoint['model_state_dict'])

checkpoint = None
if(a_cuda):
  anchors = anchors.cuda()
  yolo = yolo.cuda()
  novel = novel.cuda()
  recurrent = recurrent.cuda()

overall_yolo = []
overall_novel = []
overall_lstm = []
for i in [0,1,2,3,4,5,6,7,8,9]:
  print("i:"+str(i))
  ylabels, yimages = create_trajbatch(i, True)
  nlabels, nimages = create_diffbatch(i, True)
  llabels, _ = create_LSTMbatch(i, True)
  ntrain = yimages.shape[0]
  num_its = ntrain//batchsize
  yolo_errors = np.zeros(num_its)
  novel_errors = np.zeros(num_its)
  lstm_errors = np.zeros(num_its)

  for t in range(ntrain // batchsize):
    print("t:"+str(t))
    batchindices = np.arange(t*batchsize, (t+1)*batchsize)
    this_batch = Variable(yimages[batchindices,...].float())
    diff_batch = Variable(nimages[batchindices,...].float())
    if(a_cuda):
        this_batch = this_batch.cuda()
        diff_batch = diff_batch.cuda()

    with(torch.no_grad()):
      yout = yolo(this_batch)
      nout = novel(diff_batch)

    cyout = yout.detach().clone()
    lstm_input = shape_for_LSTM(cyout, a_cuda)
    lstm_output = recurrent(lstm_input)
    lstm_output = lstm_output.view(batchsize-1, 5, H, W)


    yout = nms_thresh(yout, 0.75, anchors, a_cuda, res)
    nout = nms_thresh(nout, 0.75, anchors, a_cuda, res)

    adj_out = match_boxes(yout, anchors, 25, (1280//2,1920//2), a_cuda)
    if(a_cuda):
      adj_out = adj_out.cuda()

    ylabel_indices = (ylabels[:, -1][..., None] == torch.tensor(batchindices)).any(-1).nonzero().squeeze()
    nlabel_indices = (nlabels[:, -1][..., None] == torch.tensor(batchindices)).any(-1).nonzero().squeeze()
    llabel_indices = (llabels[:, -1][..., None] == torch.tensor(batchindices)).any(-1).nonzero().squeeze()
    true_out = ylabels[ylabel_indices,:]
    diff_out = nlabels[nlabel_indices,:]
    lstm_out = llabels[llabel_indices,:]
    if(a_cuda):
      true_out = true_out.cuda()
      diff_out = diff_out.cuda()
      lstm_out = lstm_out.cuda()
    

    _, yolo_traj, _ = novel_train(true_out, (H,W), anchors, res, a_cuda, batchsize)
    _, novel_traj, _ = novel_train(diff_out, (H,W), anchors, res, a_cuda, batchsize)
    lstm_traj, _ = transform_LSTM_train(lstm_out, res, (H,W), a_cuda)

    yolo_errors[t] = F.mse_loss(adj_out, yolo_traj[1:,:,:2,:,:]).item()
    novel_errors[t] = F.mse_loss(nout.view(batchsize, num_anchors, 9, H, W)[:,:,5:7, :,:], novel_traj[:,:,:2,:,:])
    lstm_errors[t] = F.mse_loss(lstm_output[:,:2,:,:], lstm_traj[:,:2,:,:])

    # remove from memory
    this_batch = None
    true_out = None
    yout = None
    nout = None
    adj_out = None
    lstm_traj = None
    novel_traj = None
    gc.collect()
    if(a_cuda):
      torch.cuda.empty_cache()

  overall_yolo.append(np.mean(yolo_errors))
  overall_novel.append(np.mean(novel_errors))
  overall_lstm.append(np.mean(lstm_errors))

print(np.mean(np.array(overall_yolo)))
print(np.mean(np.array(overall_novel)))
print(np.mean(np.array(overall_lstm)))